In [3]:
%pip install requests
%pip install beautifulsoup4
%pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.0.5/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.0.5/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 26.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 33.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 7.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 27.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.0.5/libexec/bin/python -m pip install --upgrade pip
Note: you may n

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
url = 'https://www.scdf.gov.sg/firecode/table-of-content'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.content, 'html.parser')

# Find all accordion elements
accordions = soup.find_all('div', class_='firecode-accordion')

# Initialize lists to store data
chapters = []
clauses = []
contents = []

# Iterate through each accordion
for accordion in accordions:
    # Extract chapter title
    chapter_title = accordion.find('h5').text.strip()

    # Extract clauses within the accordion
    clause_elements = accordion.find_all('div', class_='clause-content-block')

    # Iterate through each clause
    for clause in clause_elements:
        # Extract individual clauses and their content
        for link in clause.find_all('a'):
            clause_title = link.text.strip()
            clause_link = link['href']

            # Append data to lists
            chapters.append(chapter_title)
            clauses.append(clause_title)
            contents.append(clause_link)

# Create a DataFrame from the lists
firecode_df = pd.DataFrame({'Chapter': chapters, 'Clause': clauses, 'Content': contents})

# Display the DataFrame
print(firecode_df.head())


                       Chapter  \
0          Chapter 1 - General   
1          Chapter 1 - General   
2          Chapter 1 - General   
3  Chapter 2 - Means of Escape   
4  Chapter 2 - Means of Escape   

                                           Clause  \
0                               Clause 1.1  Scope   
1                 Clause 1.2  Codes and Standards   
2                        Clause 1.3 Abbreviations   
3                              Clause 2.1 General   
4  Clause 2.2 Determinations of Exit Requirements   

                                             Content  
0  /firecode/table-of-content/chapter-1-general/c...  
1  /firecode/table-of-content/chapter-1-general/c...  
2  /firecode/table-of-content/chapter-1-general/c...  
3  /firecode/table-of-content/chapter-2-means-of-...  
4  /firecode/table-of-content/chapter-2-means-of-...  


In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

# Install Selenium
# Uncomment the line below if you haven't installed Selenium yet
# !pip install selenium

# Download ChromeDriver from https://sites.google.com/chromium.org/driver/
# Specify the path to the ChromeDriver executable
# chrome_driver_path = '/path/to/chromedriver'
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'


# Set up the Chrome WebDriver
driver = webdriver.Chrome(executable_path=chrome_driver_path)

# URL of the main page
url = "https://www.scdf.gov.sg/firecode/table-of-content"


# Open the main page
driver.get(url)

# Find all the chapter links
chapter_links = driver.find_elements(By.CSS_SELECTOR, ".firecode-accordion__controller--wrapper a")

# Create an empty DataFrame
firecode_df = pd.DataFrame(columns=['Chapter', 'Clause', 'Content'])

# Loop through each chapter link
for chapter_link in chapter_links:
    chapter_title = chapter_link.text.strip()

    # Click on the chapter link to reveal the clauses
    chapter_link.click()

    # Find all the clause links under the chapter
    clause_links = driver.find_elements(By.CSS_SELECTOR, ".firecode-accordion__content a")

    # Loop through each clause link
    for link in clause_links:
        clause_title = link.text.strip()
        clause_link = link.get_attribute('href')

        # Append data to DataFrame
        firecode_df = firecode_df.append({
            'Chapter': chapter_title,
            'Clause': clause_title,
            'Content': clause_link
        }, ignore_index=True)

# Close the WebDriver
driver.quit()

# Display the DataFrame
print("Unique Chapters:", firecode_df['Chapter'].unique())
print("\nDataFrame:")
print(firecode_df)


TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# URL of the main page
url = "https://www.scdf.gov.sg/firecode/table-of-content"

# Navigate to the main page
driver.get(url)

# Extracting data
firecode_data = []

# Find all chapter links
chapter_links = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion']/div[@class='firecode-accordion__controller']/div[@class='firecode-accordion__controller--wrapper']/h5/a")

for chapter_link in chapter_links:
    chapter_title = chapter_link.text.strip()
    chapter_content_link = chapter_link.get_attribute("href")

    # Append chapter data
    firecode_data.append({
        'Chapter': chapter_title,
        'Clause': None,
        'Content': chapter_content_link
    })

# Create a DataFrame
firecode_df = pd.DataFrame(firecode_data)

# Display DataFrame
print("DataFrame:")
print(firecode_df)

# Close the WebDriver
driver.quit()


DataFrame:
                                              Chapter Clause  \
0                                 Chapter 1 - General   None   
1                         Chapter 2 - Means of Escape   None   
2             Chapter 3 - Structural Fire Precautions   None   
3   Chapter 4 - Site Planning & External Firefight...   None   
4               Chapter 5 - Electrical Power Supplies   None   
5                    Chapter 6 - Firefighting Systems   None   
6   Chapter 7 - Mechanical Ventilation & Smoke Con...   None   
7   Chapter 8 - Emergency Lighting & Voice Communi...   None   
8   Chapter 9 - Additional Requirements for Each P...   None   
9   Chapter 10 - Requirements for Special Installa...   None   
10  Chapter 11 - Regulated Fire Safety Products an...   None   
11                                        Appendix 01   None   
12       Appendix 02 - Fire Safety Instruction Manual   None   

                                              Content  
0   https://www.scdf.gov.sg/firecode

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

def get_links_recursive(driver, url, links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list
    for page_link in page_links:
        link = page_link.get_attribute("href")
        links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links:
            links = get_links_recursive(driver, sub_page_link, links)
    
    return links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url)

# Display the retrieved links
for i, link in enumerate(all_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


1. https://go.gov.sg/trusted-sites
2. https://www.scdf.gov.sg/
3. https://www.scdf.gov.sg/home/about-us
4. https://www.scdf.gov.sg/home/about-us/corporate-philosophy
5. https://www.scdf.gov.sg/home/about-us/organisation-structure
6. https://www.scdf.gov.sg/home/about-us/organisation-structure/scdf-hq
7. https://www.scdf.gov.sg/home/about-us/organisation-structure/scdf-division-hq
8. https://www.scdf.gov.sg/home/about-us/organisation-structure/training-establishments
9. https://www.scdf.gov.sg/home/about-us/organisation-structure/international-co-operation
10. https://www.scdf.gov.sg/home/about-us/roles-functions
11. https://www.scdf.gov.sg/home/about-us/scdf-hall-of-fame
12. https://www.scdf.gov.sg/home/about-us/media-room
13. https://www.scdf.gov.sg/home/about-us/media-room/news
14. https://www.scdf.gov.sg/home/about-us/media-room/statistics
15. https://www.scdf.gov.sg/home/about-us/media-room/publications
16. https://www.scdf.gov.sg/home/about-us/media-room/local-major-incidents
17. 

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

def get_links_recursive(driver, url, links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list
    for page_link in page_links:
        link = page_link.get_attribute("href")
        if link.startswith("https://www.scdf.gov.sg/firecode"):
            links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links and sub_page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links = get_links_recursive(driver, sub_page_link, links)
    
    return links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url)

# Display the retrieved links
for i, link in enumerate(all_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


1. https://www.scdf.gov.sg/firecode2023/firecode2023
2. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general
3. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general/clause-1.1
4. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general/clause-1.2
5. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general/clause--1.3
6. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape
7. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape/clause-2.1
8. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape/clause-2.2
9. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape/clause-2.3
10. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape/clause-2.4
11. https://www.scdf.gov.sg/firecode/table-of-content/chapter-3-structural-fire-precuations
12. https://www.scdf.gov.sg/firecode/table-of-content/chapter-3-structural-fire-precuations/claus

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin, urlparse

def get_links_recursive(driver, url, links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list
    for page_link in page_links:
        link = page_link.get_attribute("href")
        if link.startswith("https://www.scdf.gov.sg/firecode"):
            links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links and sub_page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links = get_links_recursive(driver, sub_page_link, links)

    return links

def get_relative_links(driver, url):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all relative links on the current page
    relative_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Extract and return the relative links
    return [link.get_attribute("href") for link in relative_links if urlparse(link.get_attribute("href")).netloc == ""]

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url)

# Display the retrieved links
for i, link in enumerate(all_links, start=1):
    print(f"{i}. {link}")

    # Retrieve and display relative links within each returned link
    relative_links = get_relative_links(driver, link)
    for rel_link in relative_links:
        print(f"   - {rel_link}")

# Close the WebDriver
driver.quit()


1. https://www.scdf.gov.sg/firecode2023/firecode2023
   - tel:995
   - tel:1777
   - tel:1800 280 0000
   - tel:>1800 286 5555
2. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general
   - tel:995
   - tel:1777
   - tel:1800 280 0000
   - tel:>1800 286 5555
3. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general/clause-1.1
   - tel:995
   - tel:1777
   - tel:1800 280 0000
   - tel:>1800 286 5555
4. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general/clause-1.2
   - tel:995
   - tel:1777
   - tel:1800 280 0000
   - tel:>1800 286 5555
5. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general/clause--1.3
   - tel:995
   - tel:1777
   - tel:1800 280 0000
   - tel:>1800 286 5555
6. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape
   - tel:995
   - tel:1777
   - tel:1800 280 0000
   - tel:>1800 286 5555
7. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape/clause-2.1
   - tel:995
   

KeyboardInterrupt: 

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin, urlparse

def get_links_recursive(driver, url, links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list
    for page_link in page_links:
        link = page_link.get_attribute("href")
        if link.startswith("https://www.scdf.gov.sg/firecode"):
            links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links and sub_page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links = get_links_recursive(driver, sub_page_link, links)

    return links

def get_relative_links(driver, url):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all relative links on the current page
    relative_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Filter out telephone links
    filtered_links = [link.get_attribute("href") for link in relative_links if not link.get_attribute("href").startswith("tel:")]

    # Extract and return the relative links
    return filtered_links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url)

# Display the retrieved links
for i, link in enumerate(all_links, start=1):
    print(f"{i}. {link}")

    # Retrieve and display relative links within each returned link
    relative_links = get_relative_links(driver, link)
    for rel_link in relative_links:
        print(f"   - {rel_link}")

# Close the WebDriver
driver.quit()


1. https://www.scdf.gov.sg/firecode2023/firecode2023
   - https://go.gov.sg/trusted-sites
   - https://www.scdf.gov.sg/
   - https://www.scdf.gov.sg/home/about-us
   - https://www.scdf.gov.sg/home/about-us/corporate-philosophy
   - https://www.scdf.gov.sg/home/about-us/organisation-structure
   - https://www.scdf.gov.sg/home/about-us/organisation-structure/scdf-hq
   - https://www.scdf.gov.sg/home/about-us/organisation-structure/scdf-division-hq
   - https://www.scdf.gov.sg/home/about-us/organisation-structure/training-establishments
   - https://www.scdf.gov.sg/home/about-us/organisation-structure/international-co-operation
   - https://www.scdf.gov.sg/home/about-us/roles-functions
   - https://www.scdf.gov.sg/home/about-us/scdf-hall-of-fame
   - https://www.scdf.gov.sg/home/about-us/media-room
   - https://www.scdf.gov.sg/home/about-us/media-room/news
   - https://www.scdf.gov.sg/home/about-us/media-room/statistics
   - https://www.scdf.gov.sg/home/about-us/media-room/publications
  

KeyboardInterrupt: 

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

def get_relative_links(driver, url, exclude_keywords=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all relative links on the current page
    relative_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Filter out excluded keywords
    filtered_links = [link.get_attribute("href") for link in relative_links if not any(keyword in link.get_attribute("href") for keyword in exclude_keywords)]

    # Extract and return the relative links
    return filtered_links

def get_links_recursive(driver, url, exclude_keywords=[], links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list
    for page_link in page_links:
        link = page_link.get_attribute("href")
        if link.startswith("https://www.scdf.gov.sg/firecode"):
            links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links and sub_page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links = get_links_recursive(driver, sub_page_link, exclude_keywords, links)
    
    return links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Define the keywords to exclude
exclude_keywords = ["about-us", "ns-matters", "community-volunteers", "join-us", "payment", "rescue995", "sgfpc", "cda", "nsti", "ncdcc", "savealife", "contact-info", "facebook", "twitter", "myscdf", "report_vulnerability", "privacy-statement", "terms-of-use", "site_map", "tel:"]

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url, exclude_keywords)

# Display the retrieved links
for i, link in enumerate(all_links, start=1):
    print(f"{i}. {link}")

    # Retrieve and display filtered relative links within each returned link
    relative_links = get_relative_links(driver, link, exclude_keywords)
    for rel_link in relative_links:
        print(f"   - {rel_link}")

# Close the WebDriver
driver.quit()


1. https://www.scdf.gov.sg/firecode2023/firecode2023
   - https://go.gov.sg/trusted-sites
   - https://www.scdf.gov.sg/
   - https://www.scdf.gov.sg/home/fire-safety
   - https://www.scdf.gov.sg/home/fire-safety/home-fire-alarm-device-(hfad)
   - https://www.scdf.gov.sg/home/fire-safety/plans-and-consultations
   - https://www.scdf.gov.sg/home/fire-safety/plans-and-consultations/plan-approval
   - https://www.scdf.gov.sg/home/fire-safety/plans-and-consultations/waiver-application
   - https://www.scdf.gov.sg/home/fire-safety/plans-and-consultations/minor-addition-alteration-works
   - https://www.scdf.gov.sg/home/fire-safety/plans-and-consultations/performance-based-approach-to-fire-safety-design
   - https://www.scdf.gov.sg/home/fire-safety/plans-and-consultations/regulated-fire-safety-products
   - https://www.scdf.gov.sg/home/fire-safety/permits-and-certifications
   - https://www.scdf.gov.sg/home/fire-safety/permits-and-certifications/fire-safety-certificate-temporary-fire-permit
 

KeyboardInterrupt: 

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin, urlparse

def get_links_recursive(driver, url, links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    # Exclude links within header and footer
    header_links = driver.find_elements(By.XPATH, "//header[@class='site-header']//a[@href]")
    footer_links = driver.find_elements(By.XPATH, "//footer[@class='site-footer__wrap']//a[@href]")
    
    header_footer_links = set([link.get_attribute("href") for link in header_links + footer_links])
    
    # Find all links on the current page excluding those in header and footer
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    page_links = [link.get_attribute("href") for link in page_links if link.get_attribute("href") not in header_footer_links]

    # Append the links to the list
    for page_link in page_links:
        if page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links.append(page_link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links and sub_page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links = get_links_recursive(driver, sub_page_link, links)

    return links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url)

# Display the retrieved links
for i, link in enumerate(all_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


1. https://www.scdf.gov.sg/firecode2023/firecode2023
2. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general
3. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general/clause-1.1
4. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general/clause-1.2
5. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general/clause--1.3
6. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape
7. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape/clause-2.1
8. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape/clause-2.2
9. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape/clause-2.3
10. https://www.scdf.gov.sg/firecode/table-of-content/chapter-2-means-of-escape/clause-2.4
11. https://www.scdf.gov.sg/firecode/table-of-content/chapter-3-structural-fire-precuations
12. https://www.scdf.gov.sg/firecode/table-of-content/chapter-3-structural-fire-precuations/claus

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin, urlparse

def is_within_header_or_footer(element):
    # Check if the element or its ancestors have the class "site-header" or "site-footer__wrap"
    return element.get_attribute("class") == "site-header" or "site-footer__wrap" in element.get_attribute("class")

def get_links_recursive(driver, url, links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list, excluding those within header and footer
    for page_link in page_links:
        if not is_within_header_or_footer(page_link):
            link = page_link.get_attribute("href")
            if link.startswith("https://www.scdf.gov.sg/firecode"):
                links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links and sub_page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links = get_links_recursive(driver, sub_page_link, links)

    return links

def get_relative_links(driver, url):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all relative links on the current page, excluding those within header and footer
    relative_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Filter out links within header and footer
    filtered_links = [link.get_attribute("href") for link in relative_links if not is_within_header_or_footer(link)]

    # Extract and return the relative links
    return filtered_links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url)

# Display the retrieved links
for i, link in enumerate(all_links, start=1):
    print(f"{i}. {link}")

    # Retrieve and display relative links within each returned link
    relative_links = get_relative_links(driver, link)
    for rel_link in relative_links:
        print(f"   - {rel_link}")

# Close the WebDriver
driver.quit()



1. https://www.scdf.gov.sg/firecode2023/firecode2023
   - https://go.gov.sg/trusted-sites
   - https://www.scdf.gov.sg/
   - https://www.scdf.gov.sg/home/about-us
   - https://www.scdf.gov.sg/home/about-us/corporate-philosophy
   - https://www.scdf.gov.sg/home/about-us/organisation-structure
   - https://www.scdf.gov.sg/home/about-us/organisation-structure/scdf-hq
   - https://www.scdf.gov.sg/home/about-us/organisation-structure/scdf-division-hq
   - https://www.scdf.gov.sg/home/about-us/organisation-structure/training-establishments
   - https://www.scdf.gov.sg/home/about-us/organisation-structure/international-co-operation
   - https://www.scdf.gov.sg/home/about-us/roles-functions
   - https://www.scdf.gov.sg/home/about-us/scdf-hall-of-fame
   - https://www.scdf.gov.sg/home/about-us/media-room
   - https://www.scdf.gov.sg/home/about-us/media-room/news
   - https://www.scdf.gov.sg/home/about-us/media-room/statistics
   - https://www.scdf.gov.sg/home/about-us/media-room/publications
  

KeyboardInterrupt: 

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin, urlparse

def get_links_recursive(driver, url, links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list
    for page_link in page_links:
        link = page_link.get_attribute("href")
        if link.startswith("https://www.scdf.gov.sg/firecode"):
            links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links and sub_page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links = get_links_recursive(driver, sub_page_link, links)

    return links

def get_relative_links(driver, url, excluded_keywords=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all relative links on the current page
    relative_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Filter out specific links
    filtered_links = []
    for rel_link in relative_links:
        href = rel_link.get_attribute("href")
        if not any(keyword in href for keyword in excluded_keywords):
            filtered_links.append(href)

    return filtered_links

def is_in_header_or_footer(element):
    # Adjust this function based on the actual structure of your header and footer
    # For example, you may check for specific class names, IDs, or other patterns.
    return "site-header" in element.get_attribute("class") or "site-footer__wrap" in element.get_attribute("class")

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url)

# Display the retrieved links
for i, link in enumerate(all_links, start=1):
    print(f"{i}. {link}")

    # Retrieve and display relative links within each returned link
    excluded_keywords = [
        "about-us", "ns-matters", "community-volunteers", "join-us", "payment",
        "rescue995", "sgfpc", "cda", "nsti", "ncdcc", "savealife", "contact-info",
        "facebook", "twitter", "myscdf", "report_vulnerability", "privacy-statement",
        "terms-of-use", "site_map", "fire-safety", "civil-defence-shelter", "e-services",
        "tel", "i-am-safe", "feedback-form", "trusted-sites", "sitemap", "https://www.scdf.gov.sg/faq",
        "https://www.reach.gov.sg/", "https://www.scdf.gov.sg/firecode2023/firecode2023"
    ]
    relative_links = get_relative_links(driver, link, excluded_keywords)
    for rel_link in relative_links:
        print(f"   - {rel_link}")

# Close the WebDriver
driver.quit()


1. https://www.scdf.gov.sg/firecode2023/firecode2023
   - https://www.scdf.gov.sg/
   - https://www.scdf.gov.sg/
   - https://www.scdf.gov.sg/firecode/foreword
   - https://www.scdf.gov.sg/firecode/table-of-content
   - https://www.scdf.gov.sg/firecode2023/circulars
   - https://www.scdf.gov.sg/
2. https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general
   - https://www.scdf.gov.sg/
   - https://www.scdf.gov.sg/
   - https://www.scdf.gov.sg/firecode/table-of-content
   - https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general
   - https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general#history11
   - https://www.scdf.gov.sg/docs/default-source/scdf-library/fssd-downloads/circulars/circular-publication-of-the-code-of-practice-for-fire-precautions-in-buildings-2023-edition.pdf?sfvrsn=afe796d9_2
   - https://www.scdf.gov.sg/firecode/table-of-content/chapter-1-general#history111
   - https://www.scdf.gov.sg/docs/default-source/scdf-library/fssd-downloads

KeyboardInterrupt: 

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import os

def get_relative_links(driver, url, exclude_keywords=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all relative links on the current page
    relative_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Filter out excluded keywords
    filtered_links = [link.get_attribute("href") for link in relative_links if not any(keyword in link.get_attribute("href") for keyword in exclude_keywords)]

    # Extract and return the relative links
    return filtered_links

def get_links_recursive(driver, url, exclude_keywords=[], links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list
    for page_link in page_links:
        link = page_link.get_attribute("href")
        if link.startswith("https://www.scdf.gov.sg/firecode"):
            links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links and sub_page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links = get_links_recursive(driver, sub_page_link, exclude_keywords, links)
    
    return links

def save_page_content(driver, url, save_path):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Get the page content
    page_content = driver.page_source
    
    # Save the page content to a file
    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(page_content)

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Define the keywords to exclude
exclude_keywords = ["about-us", "ns-matters", "community-volunteers", "join-us", "payment", "rescue995", "sgfpc", "cda", "nsti", "ncdcc", "savealife", "contact-info", "facebook", "twitter", "myscdf", "report_vulnerability", "privacy-statement", "terms-of-use", "site_map"]

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url, exclude_keywords)

# Create a directory to save the pages
save_directory = '/Users/lmchak/llm/SCDFdownload'
os.makedirs(save_directory, exist_ok=True)

# Download and save each page
for i, link in enumerate(all_links, start=1):
    print(f"Downloading {i}/{len(all_links)}: {link}")
    
    # Save the main page
    save_path = os.path.join(save_directory, f"page_{i}.html")
    save_page_content(driver, link, save_path)
    
    # Retrieve and save filtered relative links within each returned link
    relative_links = get_relative_links(driver, link, exclude_keywords)
    for j, rel_link in enumerate(relative_links, start=1):
        rel_save_path = os.path.join(save_directory, f"page_{i}_sub_{j}.html")
        save_page_content(driver, rel_link, rel_save_path)

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import os
from bs4 import BeautifulSoup  # Ensure you have BeautifulSoup installed: pip install beautifulsoup4

def get_relative_links(driver, url, exclude_keywords=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all relative links on the current page
    relative_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Filter out excluded keywords
    filtered_links = [link.get_attribute("href") for link in relative_links if not any(keyword in link.get_attribute("href") for keyword in exclude_keywords)]

    # Extract and return the relative links
    return filtered_links

def get_links_recursive(driver, url, exclude_keywords=[], links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list
    for page_link in page_links:
        link = page_link.get_attribute("href")
        if link.startswith("https://www.scdf.gov.sg/firecode"):
            links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links and sub_page_link.startswith("https://www.scdf.gov.sg/firecode"):
            links = get_links_recursive(driver, sub_page_link, exclude_keywords, links)
    
    return links

def save_page_content(driver, url, save_directory):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Get the page content
    page_content = driver.page_source
    
    # Extract text within the specified span tag using BeautifulSoup
    soup = BeautifulSoup(page_content, 'html.parser')
    span_text = soup.find('span', {'itemprop': 'name'}).get_text(strip=True) if soup.find('span', {'itemprop': 'name'}) else "Unknown"
    
    # Save the page content to a file with the prefixed filename
    filename = f"{span_text}_{os.path.basename(url)}.html"
    save_path = os.path.join(save_directory, filename)
    
    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(page_content)
    
    # Return the saved path for display
    return save_path

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Define the keywords to exclude

exclude_keywords = ["about-us", "ns-matters", "community-volunteers", "join-us", "payment", "rescue995", "sgfpc", "cda", "nsti", "ncdcc", "savealife", "contact-info", "facebook", "twitter", "myscdf", "report_vulnerability", "privacy-statement", "terms-of-use", "site_map", "firesafety", "fire-safety", "civil-defence-shelter", "e-services",
        "tel", "i-am-safe", "feedback-form", "trusted-sites", "sitemap", "https://www.scdf.gov.sg/faq",
        "https://www.reach.gov.sg/", "https://www.scdf.gov.sg/firecode2023/firecode2023" ]

# Start the recursive link retrieval
main_url = "https://www.scdf.gov.sg/firecode/table-of-content/"
all_links = get_links_recursive(driver, main_url, exclude_keywords)

# Create a directory to save the pages
save_directory = '/Users/lmchak/llm/SCDFdownload'
os.makedirs(save_directory, exist_ok=True)

# Download and save each page
for i, link in enumerate(all_links, start=1):
    print(f"Downloading {i}/{len(all_links)}: {link}")
    
    # Save the main page
    save_path = save_page_content(driver, link, save_directory)
    print(f"   Saved: {save_path}")
    
    # Retrieve and save filtered relative links within each returned link
    relative_links = get_relative_links(driver, link, exclude_keywords)
    for j, rel_link in enumerate(relative_links, start=1):
        rel_save_path = save_page_content(driver, rel_link, save_directory)
        print(f"   - Saved: {rel_save_path}")

# Close the WebDriver
driver.quit()


   Saved: /Users/lmchak/llm/SCDFdownload/Fire Code 2023_firecode2023.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Unknown_.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Unknown_.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Fire Code 2023_foreword.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Fire Code 2023_table-of-content.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Fire Safety_circulars.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Unknown_.html
   Saved: /Users/lmchak/llm/SCDFdownload/Fire Code 2023_chapter-1-general.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Unknown_.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Unknown_.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Fire Code 2023_table-of-content.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Fire Code 2023_chapter-1-general.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Fire Code 2023_chapter-1-general#history11.html
   - Saved: /Users/lmchak/llm/SCDFdownload/Unknown_circular-publication-of-the-code-of-p

KeyboardInterrupt: 